<a href="https://colab.research.google.com/github/dwchangdev/ds-project/blob/main/n214a_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 1 / NOTE 4*

# 📝 Assignment
---

# Logistic Regression

이번에는 [**Cardiovascular Disease dataset**](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) 데이터셋을 사용해 봅시다. 데이터에 대한 설명을 캐글 페이지에서 잘 살펴보시고 과제를 진행해 주세요!

### 1) 데이터를 전처리 하겠습니다.
- 데이터를 불러오세요.(read_csv 사용)
- id 칼럼은 drop하세요.
- missing value를 확인하세요.
- 중복된 샘플이 있는지 확인하세요.
- **중복된 샘플을 제거하고 남은 샘플의 수를 과제 제출폼에 제출하세요.**

In [16]:
# !pip install category_encoders

In [17]:
import pandas as pd
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/cardio/cardio_train.csv', sep=';')

# id 컬럼 삭제
df.drop(columns = ['id'], inplace = True)

In [18]:
# 결측치 확인 -> 없음
df.isna().value_counts()

age    gender  height  weight  ap_hi  ap_lo  cholesterol  gluc   smoke  alco   active  cardio
False  False   False   False   False  False  False        False  False  False  False   False     70000
dtype: int64

In [19]:
# 중복된 샘플 확인
print(df[df.duplicated()==True])

         age  gender  height  weight  ap_hi  ...  gluc  smoke  alco  active  cardio
10562  20495       1     165    70.0    120  ...     1      0     0       1       0
21784  16793       1     165    68.0    120  ...     1      0     0       1       0
38505  18988       1     164    65.0    120  ...     1      0     0       1       0
40365  14552       1     158    64.0    120  ...     1      0     0       1       0
42450  18353       1     169    67.0    120  ...     1      0     0       1       0
44653  16937       2     170    70.0    120  ...     1      0     0       0       0
45125  21280       1     165    65.0    120  ...     1      0     0       1       0
45748  22077       1     175    69.0    120  ...     1      0     0       1       1
45810  21230       1     164    62.0    120  ...     1      0     0       1       0
48917  21945       1     165    60.0    120  ...     1      0     0       1       0
50432  17493       2     169    74.0    120  ...     1      0     0       1 

In [20]:
# 중복값 제거
df.drop_duplicates(inplace = True)

In [21]:
# 데이터 프레임 확인 -> 남은 샘플 개수 69,976개
df

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,22431,1,163,72.0,135,80,1,2,0,0,0,1


### 2) 다운로드 한 훈련 데이터를 훈련/검증/테스트 세트로 나누어 주세요.
- 타겟 특성을 확인하고 타겟 변수를 나누어 줍니다.
- 훈련과 테스트 데이터로 나누어 줍니다(train_test_split 사용, train/test = 80/20, random_state=2)
- 훈련데이터를 훈련과 검증데이터로 나누어 줍니다(train_test_split 사용, train/validation = 80/20, random_state=2)
- **검증 데이터(X_val)의 갯수를 과제 제출폼에 제출하세요.**

In [22]:
from sklearn.model_selection import train_test_split

features = df.drop(columns = ['cardio'])
target = df['cardio']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, train_size = 0.8, random_state = 2)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, train_size = 0.8, random_state = 2)

X_val

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
36174,20787,1,168,67.0,120,80,3,3,0,0,1
4101,20578,2,176,80.0,160,80,1,1,1,1,1
19924,18368,2,174,78.0,120,80,2,1,0,0,1
41995,17415,2,177,62.0,120,80,1,1,1,0,1
39939,23303,2,168,98.0,140,90,3,3,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
67206,16823,1,155,63.0,120,80,1,1,0,0,0
25036,22517,1,141,60.0,150,100,3,3,0,0,0
12153,15245,2,165,64.0,120,80,1,1,0,0,1
25375,18260,2,170,70.0,120,80,3,3,0,0,1


### 3) 사이킷런의 LogisticRegression을 사용해 모델을 만들어 보세요.
- 모델을 만들기 전에 기준모델을 설정합니다.
- **LogisticRegression 모델을 학습하고(max_iter=1000) 검증세트 정확도를 과제 제출폼에 제출하세요.**


In [23]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# 분류문제에서는 보통 타겟 변수에서 가장 빈번하게 나타나는 범주를 기준모델로 설정합니다.
major = y_train.mode()[0]

y_pred = [major] * len(y_train)

print("training accuracy: ", accuracy_score(y_train, y_pred))

y_pred = [major] * len(y_val)
print("validation accuracy: ", accuracy_score(y_val, y_pred))

training accuracy:  0.501697034655234
validation accuracy:  0.49821364773133264


In [24]:
logistic = LogisticRegression(max_iter = 1000)
logistic.fit(X_train, y_train)

print("검증세트 정확도 :", logistic.score(X_val, y_val))

검증세트 정확도 : 0.7073061807788495


### 4) 데이터에서 범주형 성질을 지닌 특성을 찾아 원핫인코딩 하세요.
- category_encoders를 사용해서 'gender' 특성만 원핫인코딩 하세요.
- LogisticRegression 모델을 다시 학습합니다.
- **검증세트 정확도를 과제 제출폼에 제출하세요.**

In [25]:
# 원핫인코딩
from category_encoders import OneHotEncoder

# 인코딩 객체 생성, gender 컬럼만 인코딩
encoder = OneHotEncoder(use_cat_names=True, cols="gender")

# 훈련용 데이터 fit_transform
X_train_encoded = encoder.fit_transform(X_train)

# 검증용 데이터 : transform
X_val_encoded = encoder.transform(X_val)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [26]:
# 로지스틱 회귀모델 생성 및 훈련
# 객체 생성
logistic = LogisticRegression(max_iter=1000)

# 모델 훈련 : 인코딩 완료한 데이터 사용 (X_train_encoded)
logistic.fit(X_train_encoded, y_train)

# 검증세트의 정확도 측정 : X_val_encoded, y_val사용, score 속성 사용
print('검증세트 정확도', logistic.score(X_val_encoded, y_val))

검증세트 정확도 0.6944444444444444


### 5) 학습한 모델을 검증세트에서 최적화 시킨 후, 테스트세트에서 딱 한 번 정확도를 확인해 보세요.
- **테스트 세트 정확도를 과제 제출폼에 제출하세요.(0.712 이상)**

In [27]:
# 테스트 데이터 features와 target 분리
y_test = target
X_test = features

# 확인
print("테스트용 데이터 확인 / features: {0}, target{1}".format(X_test.shape, y_test.shape))

테스트용 데이터 확인 / features: (69976, 11), target(69976,)


In [28]:
# 1) 하이퍼파라미터(C) 조정
# 객체 생성 & 모델 훈련
logistic_001 = LogisticRegression(C=0.01, max_iter=1000).fit(X_train_encoded, y_train)
logistic_01 = LogisticRegression(C=0.1, max_iter=1000).fit(X_train_encoded, y_train)
logistic = LogisticRegression(max_iter=1000).fit(X_train_encoded, y_train)
logistic_1 = LogisticRegression(C=1, max_iter=1000).fit(X_train_encoded, y_train)
logistic_10 = LogisticRegression(C=10, max_iter=1000).fit(X_train_encoded, y_train)


# 검증 데이터셋으로 정확도 측정
print('C=0.01 정확도', logistic_001.score(X_val_encoded, y_val))
print('C=0.1 정확도', logistic_01.score(X_val_encoded, y_val))
print('C=default 정확도', logistic.score(X_val_encoded, y_val))
print('C=1 정확도', logistic_1.score(X_val_encoded, y_val))
print('C=10 정확도', logistic_10.score(X_val_encoded, y_val))

C=0.01 정확도 0.6942658092175777
C=0.1 정확도 0.7094498035012504
C=default 정확도 0.6944444444444444
C=1 정확도 0.6944444444444444
C=10 정확도 0.7093604858878171


In [29]:
# 2) 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)

# 객체 생성 & 모델 훈련
logistic_001_scaled = LogisticRegression(C=0.01, max_iter=1000).fit(X_train_scaled, y_train)
logistic_01_scaled = LogisticRegression(C=0.1, max_iter=1000).fit(X_train_scaled, y_train)
logistic_scaled = LogisticRegression(max_iter=1000).fit(X_train_scaled, y_train)
logistic_1_scaled = LogisticRegression(C=1, max_iter=1000).fit(X_train_scaled, y_train)
logistic_10_scaled = LogisticRegression(C=10, max_iter=1000).fit(X_train_scaled, y_train)

# 검증 데이터셋으로 정확도 측정
print('C=0.01 정확도', logistic_001_scaled.score(X_val_scaled, y_val))
print('C=0.1 정확도', logistic_01_scaled.score(X_val_scaled, y_val))
print('C=default 정확도', logistic_scaled.score(X_val_scaled, y_val))
print('C=1 정확도', logistic_1_scaled.score(X_val_scaled, y_val))
print('C=10 정확도', logistic_10_scaled.score(X_val_scaled, y_val))

C=0.01 정확도 0.7057877813504824
C=0.1 정확도 0.721686316541622
C=default 정확도 0.723383351196856
C=1 정확도 0.723383351196856
C=10 정확도 0.7237406216505895


In [30]:
# 데이터 표준화 작업이 모델 최적화에 더욱 효과적인 것 같다.
# 테스트 데이터도 똑같이 표준화 후 모델 학습을 진행해 본다.

# test데이터의 gender 원핫인코딩
X_test_encoded = encoder.transform(X_test)

# test데이터 표준화
X_test_scaled = scaler.transform(X_test_encoded)

# test데이터를 가지고 예측 (1회) -> 정확도 UP
print('테스트 세트 정확도', logistic_scaled.score(X_test_scaled, y_test))

테스트 세트 정확도 0.7223619526694867


## 🔥 도전과제:

### 6) EDA를 추가적으로 수행하고 가능한 방법으로 모델을 튜닝해서 결과를 만들어 보세요. 최소한 다음 내용들이 포함이 되어야 합니다.
- 새로운 특성을 최소한 한 개 이상 만드세요.
- 특성들을 확인해서 아웃라이어라고 생각되는 데이터를 가진 샘플을 제거하세요.
- [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) 을 사용하세요. (여러가지 종류의 Scalar를 어떤 상황에 적용해야 하는지 알아보고 사용해 보세요.)
- 만들어진 모델 성능을 리포팅하고 해석합니다.
